# 分词

中文分词模型，提供目前比较常见的分词模型，结合代码进行实现。主要有词典(也叫机械分词)、N_gram、隐马尔可夫HMM、条件随机场CRF几种模型的分词。

## 数据


  ● 数据集来自SIGHAN，SIGHAN 是国际计算语言协会ACL中文处理小组的简称。目前SIGHAN bakeoff 已经举办了 6 届，其中语料资源免费。选用 icwb-data2 数据作为数据集。
  
  ● icwb-data2 中包含train、test、scripts、gold、doc 目录
  
      ○ doc：数据集的一些使用指南
      ○ training： 包含已经分词的训练数据集目录。这里选择 msr_training.utf8 作为训练集。其他信息可见doc目录下的说明
          ■ 文件后缀名为 utf8 的表示编码格式为 UTF-8。
          ■ 文件前缀 msr_ ，代表是微软亚洲研究院提供。
      ○ testing：未切分的测试数据集
      ○ scripts：评分脚本和简单的分词器
      ○ gold：测试数据集的标准分词和训练集中抽取的词表
      
  ● 数据集下载：http://sighan.cs.uchicago.edu/bakeoff2005/

In [1]:
!head -300 ../data/icwb2-data/training/msr_training.utf8

训练数据集中有的数据存在 “，‘ 多余字符。可在预处理中剔除。

In [2]:
!head -10 ../data/icwb2-data/testing/msr_test.utf8

### 数据预处理

读出文本数据。将数据集装出成字符格式。

In [3]:
train_file = "../data/icwb2-data/training/msr_training.utf8"
test_file = "../data/icwb2-data/testing/msr_test.utf8"

In [4]:
import sys

sys.path.append("../")
from module.core.data_tools import DataTools

def handle(line_data):
    
    new_data = list()
    # 以空格字符划分数据。
    for word in line_data.strip().split(' '):
        # 剔除空格字符
        if word == ' ' or word == '' or len(word) == 0:
            continue
        
        new_data.append(word)
    
    return new_data
    
train_data = DataTools.Preprocess.read_file_data(train_file, del_start_str=["“", "’"], handle_func=handle)
test_data = DataTools.Preprocess.read_file_data(test_file)

86924it [00:01, 46762.12it/s]
3985it [00:00, 152823.46it/s]


In [5]:
print("train dataset: ")
for data in train_data[:100]:
    print(data)
    
print()

print("test dataset: ")
for data in test_data[:100]:
    print(data)

train dataset: 
['人们', '常', '说', '生活', '是', '一', '部', '教科书', '，', '而', '血', '与', '火', '的', '战争', '更', '是', '不可多得', '的', '教科书', '，', '她', '确实', '是', '名副其实', '的', '‘', '我', '的', '大学', '’', '。']
['心', '静', '渐', '知', '春', '似', '海', '，', '花', '深', '每', '觉', '影', '生', '香', '。']
['吃', '屎', '的', '东西', '，', '连', '一', '捆', '麦', '也', '铡', '不', '动', '呀', '？']
['他', '“', '严格要求', '自己', '，', '从', '一个', '科举', '出身', '的', '进士', '成为', '一个', '伟大', '的', '民主主义', '者', '，', '进而', '成为', '一', '位', '杰出', '的', '党外', '共产主义', '战士', '，', '献身', '于', '崇高', '的', '共产主义', '事业', '。']
['征', '而', '未', '用', '的', '耕地', '和', '有', '收益', '的', '土地', '，', '不准', '荒芜', '。']
['这', '首先', '是', '个', '民族', '问题', '，', '民族', '的', '感情', '问题', '。']
['我', '扔', '了', '两颗', '手榴弹', '，', '他', '一下子', '出', '溜', '下去', '。']
['废除', '先前', '存在', '的', '所有制', '关系', '，', '并不是', '共产主义', '所', '独具', '的', '特征', '。']
['这个', '案子', '从', '始', '至今', '我们', '都', '没有', '跟', '法官', '接触', '过', '，', '也', '没有', '跟', '原告', '、', '被告', '接触', '过', '。']
['你', '只有', '把', '事情', '做

## 词典分词

关于分词的介绍参考个人云笔记：http://note.youdao.com/noteshare?id=81e18f06e7c39d59da74323cc5aff346

词典分词模型已封装在 module/segmentation/dict/dict_segmentation.py 类中。详细代码在可在此文件中查看。模型提供以下函数

 1. 拟合：根据提供的训练数据创建词典。 并进行保存
 2. 评估：读取测试数据，并对数据进行分词，写入文本
 3. 分词：输入某个句子文本，对其进行分词
 4. 添加新词：对与某个句子中的词汇没有正确识别，可通过添加新词方式实现。
 5. 加载模型：对拟合保存的模型进行加载

In [18]:
from module.segmentation.dict.dict_segmentation import DictSegmentation

In [19]:
# 定义词的最大匹配长度。用于在正向匹配和逆向匹配中对词进行划分。
max_matching = 10 

# 初始化对象
dict_seg = DictSegmentation(max_matching=max_matching)

In [20]:
save_model = '../model/dict_segmentation.pickle'

dict_seg.fit(train_data, save_model=save_model)

100%|██████████| 86924/86924 [00:00<00:00, 95647.12it/s]

save dictionary success! File: ../model/dict_segmentation.pickle
word count： 88136


In [21]:
result = '../result/dict/msr_test_seg_result.utf8'

dict_seg.eval(test_data, seg_lab="  ", w_file=result, threads=3)

thread_0 start...
thread_1 start...
thread_2 start...
thread_0 Process:1328/1328 	thread_1 Process:1328/1328 	thread_2 Process:1329/1329 	Total process: 3985/3985 Percentage:100.00%

over！File: ../result/dict/msr_test_seg_result.utf8, encoding: utf-8


#### 评分

使用下载数据集中提供的 scripts 评分脚本对测试数据集进行评分。详细查看 README .  下面摘自README 。

* Scoring

The script 'score' is used to generate compare two segmentations. The
script takes three arguments:

1. The training set word list
2. The gold standard segmentation
3. The segmented test file

You must not mix character encodings when invoking the scoring
script. For example:

% perl scripts/score gold/cityu_training_words.utf8 gold/cityu_test_gold.utf8 test_segmentation.utf8 > score.ut8

In [22]:
!perl ../data/icwb2-data/scripts/score ../data/icwb2-data/gold/msr_training_words.utf8 ../data/icwb2-data/gold/msr_test_gold.utf8 ../result/dict/msr_test_seg_result.utf8 > ../result/dict/score.utf8

查看评分结果。评分结果输入到 ..／result/dict_seg_score.utf8 文件中. 结果在最后几行中。

In [23]:
!tail -22 ../result/dict/score.utf8

INSERTIONS:	0
DELETIONS:	4
SUBSTITUTIONS:	4
NCHANGE:	8
NTRUTH:	45
NTEST:	41
TRUE WORDS RECALL:	0.822
TEST WORDS PRECISION:	0.902
=== SUMMARY:
=== TOTAL INSERTIONS:	5090
=== TOTAL DELETIONS:	385
=== TOTAL SUBSTITUTIONS:	4196
=== TOTAL NCHANGE:	9671
=== TOTAL TRUE WORD COUNT:	106873
=== TOTAL TEST WORD COUNT:	111578
=== TOTAL TRUE WORDS RECALL:	0.957
=== TOTAL TEST WORDS PRECISION:	0.917
=== F MEASURE:	0.937
=== OOV Rate:	0.026
=== OOV Recall Rate:	0.025
=== IV Recall Rate:	0.982
###	../result/dict/msr_test_seg_result.utf8	5090	385	4196	9671	106873	111578	0.957	0.917	0.937	0.026	0.025	0.982


### 分词

In [24]:
sentence = "我爱吃雪糕。"
dict_seg.cut(sentence, seg_lab="/")

'我/爱/吃/雪/糕/。'

### 增加新词

In [25]:
dict_seg.add_word(["雪糕"], is_save=True, model_file=save_model)
dict_seg.cut(sentence, seg_lab="/")

save dictionary success! File: ../model/dict_segmentation.pickle


'我/爱/吃/雪糕/。'

### 加载模型

In [26]:
d_seg = dict_seg.load(save_model, max_matching=max_matching)

print(d_seg.cut("扬帆远东做与中国合作的先行", seg_lab="/"))
print(d_seg.cut("我爱吃雪糕。", seg_lab="/"))

扬帆/远东/做/与/中国/合作/的/先行
我/爱/吃/雪糕/。


## N-gram 分词



ngram分词模型已封装在 module/segmentation/ngram/ngram_segmentation.py 类中。详细代码在可在此文件中查看。模型提供以下函数

 1. 拟合：根据提供的训练数据创建词典。 并进行保存
 2. 评估：读取测试数据，并对数据进行分词，写入文本
 3. 分词：输入某个句子文本，对其进行分词
 4. 加载模型：对拟合保存的模型进行加载

### 加载库

In [27]:
from module.segmentation.ngram.ngram_segmentation import NgramSegmentation

### 初始化
 
 备注：n = {2, 3} n = 3 的时候需要足够大的内存，否则容易出现内存错误

In [29]:
n = 2
max_matching = 10

ngram_seg = NgramSegmentation(n, max_matching)

### 拟合

In [30]:
ngram_seg.fit(train_data)

100%|██████████| 86924/86924 [00:07<00:00, 11977.07it/s]

word number:  88139 total words number: 2541066


### 评估

In [32]:
result = "../result/ngram/msr_test_seg_result.utf8"

ngram_seg.eval(test_data, seg_lab="  ", w_file=result, threads=3)

thread_0 start...thread_1 start...

thread_2 start...
thread_0 Process:1328/1328 	thread_1 Process:1328/1328 	thread_2 Process:1329/1329 	Total process: 3985/3985 Percentage:100.00%

over！File: ../result/ngram/msr_test_seg_result.utf8, encoding: utf-8


#### 评分

使用下载数据集中提供的 scripts 评分脚本对测试数据集进行评分。（同上）

In [33]:
!perl ../data/icwb2-data/scripts/score ../data/icwb2-data/gold/msr_training_words.utf8 ../data/icwb2-data/gold/msr_test_gold.utf8 ../result/ngram/msr_test_seg_result.utf8 > ../result/ngram/score.utf8

In [34]:
!tail -22 ../result/ngram/score.utf8

INSERTIONS:	0
DELETIONS:	4
SUBSTITUTIONS:	4
NCHANGE:	8
NTRUTH:	45
NTEST:	41
TRUE WORDS RECALL:	0.822
TEST WORDS PRECISION:	0.902
=== SUMMARY:
=== TOTAL INSERTIONS:	4954
=== TOTAL DELETIONS:	389
=== TOTAL SUBSTITUTIONS:	3567
=== TOTAL NCHANGE:	8910
=== TOTAL TRUE WORD COUNT:	106873
=== TOTAL TEST WORD COUNT:	111438
=== TOTAL TRUE WORDS RECALL:	0.963
=== TOTAL TEST WORDS PRECISION:	0.924
=== F MEASURE:	0.943
=== OOV Rate:	0.026
=== OOV Recall Rate:	0.025
=== IV Recall Rate:	0.988
###	../result/ngram/msr_test_seg_result.utf8	4954	389	3567	8910	106873	111438	0.963	0.924	0.943	0.026	0.025	0.988


### 分词

In [35]:
ngram_seg.cut("扬帆远东做与中国合作的先行", seg_lab="/")

'扬帆/远东/做/与/中国/合作/的/先行'

## HMM(隐马尔可夫) 分词

hmm分词模型已封装在 module/segmentation/hmm/hmm_segmentation.py 类中。详细代码在可在此文件中查看。模型提供以下函数

1. fit 拟合模型。输入训练数据集。创建发射概率矩阵和转移概率矩阵
2. eval 评估。输入测试数据集。对测试数据进行分词，将分词结果写入文本
3. cut 分词。输入文本，对文本进行分词
4. load 加载模型。加载训练好的模型s

### 加载库

In [37]:
from module.segmentation.hmm.hmm_segmentation import HmmSegmentation

### 初始化

In [38]:
hmm_seg = HmmSegmentation()

### 拟合

In [39]:
save_model = "../model/hmm_segmentation.pickle"

hmm_seg.fit(train_data, save_model=save_model)

100%|██████████| 5167/5167 [00:00<00:00, 582595.47it/s]
86924it [00:04, 21269.07it/s]
100%|██████████| 86924/86924 [00:04<00:00, 21098.48it/s]


word number:  5167
save dictionary success! File: ../model/hmm_segmentation.pickle


### 评估


In [40]:
result = "../result/hmm/msr_test_seg_result.utf8"

hmm_seg.eval(test_data, seg_lab="  ", w_file=result, threads=3)

thread_0 start...
thread_1 start...
thread_2 start...
thread_0 Process:1328/1328 	thread_1 Process:1328/1328 	thread_2 Process:1329/1329 	Total process: 3985/3985 Percentage:100.00%

over！File: ../result/hmm/msr_test_seg_result.utf8, encoding: utf-8


#### 评分

使用下载数据集中提供的 scripts 评分脚本对测试数据集进行评分。（同上）

In [41]:
!perl ../data/icwb2-data/scripts/score ../data/icwb2-data/gold/msr_training_words.utf8 ../data/icwb2-data/gold/msr_test_gold.utf8 ../result/hmm/msr_test_seg_result.utf8 > ../result/hmm/score.utf8

In [42]:
!tail -22 ../result/hmm/score.utf8

INSERTIONS:	2
DELETIONS:	3
SUBSTITUTIONS:	5
NCHANGE:	10
NTRUTH:	45
NTEST:	44
TRUE WORDS RECALL:	0.822
TEST WORDS PRECISION:	0.841
=== SUMMARY:
=== TOTAL INSERTIONS:	7258
=== TOTAL DELETIONS:	5710
=== TOTAL SUBSTITUTIONS:	17360
=== TOTAL NCHANGE:	30328
=== TOTAL TRUE WORD COUNT:	106873
=== TOTAL TEST WORD COUNT:	108421
=== TOTAL TRUE WORDS RECALL:	0.784
=== TOTAL TEST WORDS PRECISION:	0.773
=== F MEASURE:	0.778
=== OOV Rate:	0.026
=== OOV Recall Rate:	0.362
=== IV Recall Rate:	0.796
###	../result/hmm/msr_test_seg_result.utf8	7258	5710	17360	30328	106873	108421	0.784	0.773	0.778	0.026	0.362	0.796


### 分词

In [43]:
hmm_seg.cut("扬帆远东做与中国合作的先行", seg_lab="/")

'扬帆/远东/做/与/中国/合作/的/先行'

### 加载模型

    def load(model_file):
        """
        加载模型
        :param model_file: <str> 模型文件
        :return: <HmmSegmentation> 分词模型
        """

In [9]:
save_model = "../model/hmm_segmentation.pickle"

h_mm = HmmSegmentation.load(save_model)
h_mm.cut("扬帆远东做与中国合作的先行", seg_lab="/")

'扬帆/远东/做/与/中国/合作/的/先行'

## CRF 分词

  ● CRF 也叫条件随机场，解决了在 HMM 分词问题上不能做特征选择的问题。同时采用全局归一化，解决了最大熵隐马尔可夫模型出现的标注偏置的问题。
  
  ● CRF 优缺点：
  
      ○ 优点：文字词语出现的频率信息，同时考虑上下文语境，具备较好的学习能力，因此其对歧义词和未登录词的识别都具有良好的效果
      ○ 缺点：训练周期较长，运营时计算量较大，性能不如词典分词
  
  ● 相关开源实现：
  
      ○ CRF++。  目前普遍认为比较好的分词工具包。但是目前没有可调用的API，只能根据提供的脚步使用。
          ■ 相关中文分词参考：51nlp 比较详细的说明了CRF++分词的操作说明。
          ■ CRF++ 文档
      ○ Genius。https://github.com/duanhongyi/genius
      ○ sklean_crfsuite.CRF: https://sklearn-crfsuite.readthedocs.io/en/latest/api.html

模型使用 [sklearn_crfsuite](https://sklearn-crfsuite.readthedocs.io/en/latest/api.html) 进行构建。详细参考 module/segmentation/crf/crf_segmentation.py

### 加载库

In [7]:
from module.segmentation.crf.crf_segmentation import CRFSegmentation

### 初始化

    def __init__(self, algorithm='lbfgs', min_freq=0, c1=0, c2=1.0, max_iterations=None):
        """
        选用 sklearn_crfsuite AIP 文档中常用参数。详细的参数信息，可以参考 sklearn_crfsuite API 文档。
        sklearn_crfsuite API 文档：https://sklearn-crfsuite.readthedocs.io/en/latest/api.html
        """

In [8]:
crf = CRFSegmentation()

### 拟合


In [9]:
save_model = "../model/crf_segmentation.pickle"

crf.fit(train_data, save_model=save_model)

100%|██████████| 86924/86924 [00:09<00:00, 8866.51it/s] 


Save model over! File: ../model/crf_segmentation.pickle


### 评估


In [11]:
result = "../result/crf/msr_test_seg_result.utf8"

crf.eval(test_data, seg_lab="  ", w_file=result)

thread_0 start...
thread_1 start...
thread_2 start...
thread_0 Process:1328/1328 	thread_1 Process:1328/1328 	thread_2 Process:1329/1329 	Total process: 3985/3985 Percentage:100.00%

over！File: ../result/crf/msr_test_seg_result.utf8, encoding: utf-8


#### 评分

使用下载数据集中提供的 scripts 评分脚本对测试数据集进行评分。（同上）

In [12]:
!perl ../data/icwb2-data/scripts/score ../data/icwb2-data/gold/msr_training_words.utf8 ../data/icwb2-data/gold/msr_test_gold.utf8 ../result/crf/msr_test_seg_result.utf8 > ../result/crf/score.utf8

In [13]:
!tail -22 ../result/crf/score.utf8

INSERTIONS:	2
DELETIONS:	1
SUBSTITUTIONS:	8
NCHANGE:	11
NTRUTH:	45
NTEST:	46
TRUE WORDS RECALL:	0.800
TEST WORDS PRECISION:	0.783
=== SUMMARY:
=== TOTAL INSERTIONS:	4523
=== TOTAL DELETIONS:	4782
=== TOTAL SUBSTITUTIONS:	11265
=== TOTAL NCHANGE:	20570
=== TOTAL TRUE WORD COUNT:	106873
=== TOTAL TEST WORD COUNT:	106614
=== TOTAL TRUE WORDS RECALL:	0.850
=== TOTAL TEST WORDS PRECISION:	0.852
=== F MEASURE:	0.851
=== OOV Rate:	0.026
=== OOV Recall Rate:	0.586
=== IV Recall Rate:	0.857
###	../result/crf/msr_test_seg_result.utf8	4523	4782	11265	20570	106873	106614	0.850	0.852	0.851	0.026	0.586	0.857


### 分词


In [14]:
sent = "种田要有个明白账，投本要赚利润是起码的道理。"

crf.cut(sent, seg_lab="/")

'种田/要/有/个明/白账/，/投本/要/赚/利润/是/起码/的/道理/。'

### 加载模型



In [15]:
model = "../model/crf_segmentation.pickle"

crf_model = CRFSegmentation.load(model)
crf_model.cut(sent, seg_lab="/")

'种田/要/有/个明/白账/，/投本/要/赚/利润/是/起码/的/道理/。'